## You're here! 
Welcome to your first competition in the [ITI's AI Pro training program](https://ai.iti.gov.eg/epita/ai-engineer/)! We hope you enjoy and learn as much as we did prepairing this competition.


## Introduction

In the competition, it's required to predict the `Severity` of a car crash given info about the crash, e.g., location.

This is the getting started notebook. Things are kept simple so that it's easier to understand the steps and modify it.

Feel free to `Fork` this notebook and share it with your modifications **OR** use it to create your submissions.

### Prerequisites
You should know how to use python and a little bit of Machine Learning. You can apply the techniques you learned in the training program and submit the new solutions! 

### Checklist
You can participate in this competition the way you perefer. However, I recommend following these steps if this is your first time joining a competition on Kaggle.

* Fork this notebook and run the cells in order.
* Submit this solution.
* Make changes to the data processing step as you see fit.
* Submit the new solutions.

*You can submit up to 5 submissions per day. You can select only one of the submission you make to be considered in the final ranking.*


Don't hesitate to leave a comment or contact me if you have any question!

## Import the libraries

We'll use `pandas` to load and manipulate the data. Other libraries will be imported in the relevant sections.

In [ ]:
import pandas as pd
import os

## Exploratory Data Analysis
In this step, one should load the data and analyze it. However, I'll load the data and do minimal analysis. You are encouraged to do thorough analysis!

Let's load the data using `pandas` and have a look at the generated `DataFrame`.

In [ ]:
dataset_path = '/kaggle/input/car-crashes-severity-prediction/'

training_df = pd.read_csv(os.path.join(dataset_path, 'train.csv'))
training_df= training_df.replace({False:0,True :1,'R':0,'L':1})
for i,val in enumerate(training_df['timestamp']):
    val = training_df['timestamp'][i].split(':')[0]
    training_df['timestamp'][i]=val
training_df.drop(['Roundabout','No_Exit','Give_Way','Bump','Side','Railway'],axis=1,inplace=True)
training_df.isna().sum()
training_df






In [ ]:
weather_df = pd.read_csv(os.path.join(dataset_path, 'weather-sfcsv.csv'), converters={'Month': lambda x: str(x), 'Day':lambda x: str(x), 'Hour':lambda x: str(x)})

weather_df["timestamp"] = weather_df["Year"].astype(str) +"-"+ weather_df["Month"].astype(str)+"-"+ weather_df["Day"].astype(str)+" "+ weather_df["Hour"].astype(str)
weather_df = weather_df.drop_duplicates(subset = ["timestamp"])
weather_df["Weather_Condition"] = weather_df["Weather_Condition"] .astype('category')
weather_df["Weather_Condition"] = weather_df["Weather_Condition"].cat.codes
weather_df.drop(columns=['Wind_Chill(F)','Precipitation(in)','Wind_Speed(mph)','Year','Month','Day','Hour','Selected','Visibility(mi)'],inplace=True)
weather_df




In [ ]:
import pandas as pd
  
# using merge function by setting how='inner'
final_dataframe = pd.merge(training_df, weather_df, 
                   on='timestamp', 
                   how='left')

final_dataframe.dropna(inplace=True)

#final_dataframe = final_dataframe.fillna(final_dataframe.median())
final_dataframe = final_dataframe.drop_duplicates(subset = ["timestamp"])
final_dataframe['Distance(mi)']=final_dataframe['Distance(mi)']*1000





  
# displaying result
final_dataframe

final_dataframeWe've got 6407 examples in the dataset with 14 featues, 1 ID, and the `Severity` of the crash.

By looking at the features and a sample from the data, the features look of numerical and catogerical types. What about some descriptive statistics?

In [ ]:
final_dataframe.drop(columns=['timestamp','ID'],inplace=True)

The output shows desciptive statistics for the numerical features, `Lat`, `Lng`, `Distance(mi)`, and `Severity`. I'll use the numerical features to demonstrate how to train the model and make submissions. **However you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.**

## Data Splitting

Now it's time to split the dataset for the training step. Typically the dataset is split into 3 subsets, namely, the training, validation and test sets. In our case, the test set is already predefined. So we'll split the "training" set into training and validation sets with 0.8:0.2 ratio. 

*Note: a good way to generate reproducible results is to set the seed to the algorithms that depends on randomization. This is done with the argument `random_state` in the following command* 

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(final_dataframe, test_size=0.2, random_state=42) # Try adding `stratify` here

X_train = train_df.drop(columns=[ 'Severity'])
y_train = train_df['Severity']

X_val = val_df.drop(columns=[ 'Severity'])
y_val = val_df['Severity']

X_train


As pointed out eariler, I'll use the numerical features to train the classifier. **However, you shouldn't use the numerical features only to make the final submission if you want to make it to the top of the leaderboard.** 

In [ ]:
# This cell is used to select the numerical features. IT SHOULD BE REMOVED AS YOU DO YOUR WORK.
#X_train = X_train[['Lat', 'Lng', 'Distance(mi)']]
#X_val = X_val[['Lat', 'Lng', 'Distance(mi)']]

## Model Training

Let's train a model with the data! We'll train a Random Forest Classifier to demonstrate the process of making submissions. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=2, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)

Now let's test our classifier on the validation dataset and see the accuracy.

In [ ]:
print("The accuracy of the classifier on the validation set is ", (classifier.score(X_val, y_val)))

Well. That's a good start, right? A classifier that predicts all examples' `Severity` as 2 will get around 0.63. You should get better score as you add more features and do better data preprocessing.

## Submission File Generation

We have built a model and we'd like to submit our predictions on the test set! In order to do that, we'll load the test set, predict the class and save the submission file. 

First, we'll load the data.

In [ ]:
test_df = pd.read_csv(os.path.join(dataset_path, 'test.csv'))
test_df.drop(['Roundabout','No_Exit','Give_Way','Bump','Side','Railway'],axis=1,inplace=True)

test_df.head()
ID1=test_df['ID'][0]
test_df

In [ ]:
X_test = test_df.drop(columns=['ID'])
X_test= X_test.replace({False:0,True :1,'R':0,'L':1})
for i,val in enumerate(X_test['timestamp']):
    val = X_test['timestamp'][i].split(':')[0]
    X_test['timestamp'][i]=val
finaltest_dataframe = pd.merge(X_test, weather_df, 
                   on='timestamp', 
                   how='left')

#finaltest_dataframe.dropna(inplace = True)
#finaltest_dataframe = finaltest_dataframe.drop_duplicates(subset = ["timestamp"])
finaltest_dataframe.drop(['timestamp'],axis=1,inplace=True)

#finaltest_dataframe = finaltest_dataframe.iloc[0:1601,:]

# displaying result
finaltest_dataframe

Note that the test set has the same features and doesn't have the `Severity` column.
At this stage one must **NOT** forget to apply the same processing done on the training set on the features of the test set.

Now we'll add `Severity` column to the test `DataFrame` and add the values of the predicted class to it.

**I'll select the numerical features here as I did in the training set. DO NOT forget to change this step as you change the preprocessing of the training data.**

In [ ]:

# You should update/remove the next line once you change the features used for training

y_test_predicted = classifier.predict(finaltest_dataframe)

finaltest_dataframe['Severity'] = y_test_predicted

finaltest_dataframe

In [ ]:
finaltest_dataframe["ID"] = finaltest_dataframe.index + ID1
finaltest_dataframe


Now we're ready to generate the submission file. The submission file needs the columns `ID` and `Severity` only.

In [ ]:
finaltest_dataframe[['ID', 'Severity']].to_csv('/kaggle/working/submission.csv', index=False)

The remaining steps is to submit the generated file and are as follows. 

1. Press `Save Version` on the upper right corner of this notebook.
2. Write a `Version Name` of your choice and choose `Save & Run All (Commit)` then click `Save`.
3. Wait for the saved notebook to finish running the go to the saved notebook.
4. Scroll down until you see the output files then select the `submission.csv` file and click `Submit`.

Now your submission will be evaluated and your score will be updated on the leaderboard! CONGRATULATIONS!!

## Conclusion

In this notebook, we have demonstrated the essential steps that one should do in order to get "slightly" familiar with the data and the submission process. We chose not to go into details in each step to keep the welcoming notebook simple and make a room for improvement.

You're encourged to `Fork` the notebook, edit it, add your insights and use it to create your submission.